In [1]:
!pip -q install gradio_client

In [ ]:
import os, re, time, shutil, datetime
from gradio_client import Client

# ----------------------------
# Config
# ----------------------------
SPACE_IDS = [
    "zerogpu-aoti/wan2-2-fp8da-aoti",                        # owner/space id form
    "https://zerogpu-aoti-wan2-2-fp8da-aoti.hf.space/",      # direct URL form
]

NEGATIVE_PROMPT = (
    "色调艳丽, 过曝, 静态, 细节模糊不清, 字幕, 风格, 作品, 画作, 画面, 静止, 整体发灰, 最差质量, 低质量, "
    "JPEG压缩残留, 丑陋的, 残缺的, 多余的手指, 画得不好的手部, 画得不好的脸部, 畸形的, 毁容的, "
    "形态畸形的肢体, 手指融合, 静止不动的画面, 杂乱的背景, 三条腿, 背景人很多, 倒着走"
)

# Optional: If the Space is gated, put a HF token in an env var in Colab before running:
# %env HF_TOKEN=hf_********************************
HF_TOKEN = os.getenv("HF_TOKEN", None)

# Drive output or Colab output
OUT_DIR = "/content/videos"
try:
    # If you set USE_DRIVE=True in the previous cell, prefer a Drive folder
    if os.path.ismount("/content/drive"):
        OUT_DIR = "/content/drive/MyDrive/t2v_videos"
except Exception:
    pass
os.makedirs(OUT_DIR, exist_ok=True)

PRESET_PROMPTS = {
    "1": "Demonstration of nailing a standard nail into a block of wood using a claw hammer.",
    "2": "Demonstration of installing a screw into a block of wood using a power screwdriver, followed by uninstalling (unscrewing) the screw.",
    "3": "Adding gas from a gas can to a vehicle."
}

# ----------------------------
# Helpers
# ----------------------------
def sanitize_filename(s: str) -> str:
    s = re.sub(r'[^A-Za-z0-9_. -]+', '_', s)[:80]
    return s.strip().replace(' ', '_')

def connect(space_ids, hf_token=None) -> Client:
    last_err = None
    for target in space_ids:
        try:
            print(f"Connecting to: {target}")
            c = Client(target, hf_token=hf_token) if hf_token else Client(target)
            # Force fetch API schema to fail fast if something's wrong
            _ = c.view_api(return_format="dict")
            print("✓ Connected.")
            return c
        except Exception as e:
            print(f"Connection failed: {e}")
            last_err = e
    raise last_err

def generate_video(
    client: Client,
    prompt: str,
    out_dir: str = OUT_DIR,
    duration: float = 7.1,
    guidance_scale: float = 1.0,
    guidance_scale_2: float = 3.0,
    steps: float = 4.0,
    seed: float = 42.0,
    randomize_seed: bool = True,
    retries: int = 4
) -> str:
    """
    Calls the Space and saves the returned MP4 to out_dir.
    Includes duration fallback if the Space reports GPU quota limits.
    """
    os.makedirs(out_dir, exist_ok=True)
    durations_try = [duration, 5.0, 4.0, 3.5]  # fallback durations
    last_exc = None

    for attempt in range(min(retries, len(durations_try))):
        try:
            d = float(durations_try[attempt])
            print(f"Attempt {attempt+1}: duration={d}s")
            res = client.predict(
                prompt=prompt,
                negative_prompt=NEGATIVE_PROMPT,
                duration_seconds=d,
                guidance_scale=float(guidance_scale),
                guidance_scale_2=float(guidance_scale_2),
                steps=float(steps),
                seed=float(seed),
                randomize_seed=bool(randomize_seed),
                api_name="/generate_video",
            )

            # res is (dict(video: filepath, subtitles: filepath | None), seed float)
            out = res[0]
            video_path = out.get("video") if isinstance(out, dict) else None
            if not video_path or not os.path.exists(video_path):
                raise RuntimeError(f"Server returned but video file not found. Raw: {out}")

            ts = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
            basename = f"{ts}_{sanitize_filename(prompt[:60])}.mp4"
            dest = os.path.join(out_dir, basename)
            shutil.copy(video_path, dest)
            print(f"✓ Saved: {dest}")
            return dest

        except Exception as e:
            last_exc = e
            msg = str(e)
            print(f"Error: {msg}")
            # If GPU quota / time limit, automatically try the next (shorter) duration.
            if attempt < retries - 1:
                # modest backoff to be polite to the Space
                time.sleep(2 + attempt * 2)

    # If all attempts failed, re-raise the last error
    raise last_exc

def choose_prompt() -> str:
    print("\nChoose a scene:")
    print("  1) Nail with hammer")
    print("  2) Screw in & out with power screwdriver")
    print("  3) Add gas from gas can to vehicle")
    print("  4) Free-form (type your own)  <-- bonus")
    choice = input("Enter 1, 2, 3, or 4: ").strip()
    if choice in PRESET_PROMPTS:
        return PRESET_PROMPTS[choice]
    elif choice == "4":
        custom = input("Describe the scene you want: ").strip()
        if not custom:
            raise SystemExit("No prompt given.")
        return custom
    else:
        raise SystemExit("Invalid choice.")

# ----------------------------
# Run
# ----------------------------
client = connect(SPACE_IDS, HF_TOKEN)
prompt = choose_prompt()
print(f"\nGenerating for:\n{prompt}\n")
mp4_path = generate_video(client, prompt)

# Show inline in Colab
try:
    from IPython.display import Video, display
    display(Video(mp4_path, embed=True, width=720))
except Exception:
    pass

print("\nAll done.")
print(f"Your video is here: {mp4_path}")
print("Tip: Use the Files sidebar in Colab (or your Drive folder) to download.")
